<a href="https://colab.research.google.com/github/justinballas/Finance-Projects/blob/main/Ratios_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import requests
import datetime as dt
API_KEY = ''
years = 5
limit = 5*4
#information is form financial modelling prep api

In [2]:
sp500 = requests.get('https://financialmodelingprep.com/api/v3/sp500_constituent?apikey={}'.format(API_KEY))
sp500 = sp500.json()
sp500 = pd.DataFrame(sp500)
sp500_list = list(sp500['symbol'])

In [87]:
#looping through the list of tickers to get financial statements for each ticker
income_statement = [requests.get('https://financialmodelingprep.com/api/v3/income-statement/'+
                                 '{}?period=quarter&limit={}&apikey={}'.format(symbol, limit, API_KEY)) for symbol in sp500_list]
                                 
#converting the json object for each ticker
income_statement = [statement.json() for statement in income_statement]

#turning the list of json outputs into a dataframe
income_statement = pd.concat(pd.DataFrame(income_statement[i]) for i in range(len(income_statement)))

#converting the items in the date column into datetime objects
income_statement['date'] = [dt.datetime.strptime(date, "%Y-%m-%d") for date in income_statement['date']]

#creating the period column
income_statement.reset_index(inplace=True)
income_statement['year'] = [income_statement['date'][i].strftime("%Y") for i in range(len(income_statement['period']))]

#setting the multi index
income_statement.set_index(['symbol', 'year', 'period'], inplace=True)

Repeating for balance sheet

In [89]:
#looping through the list of tickers to get financial statements for each ticker
balance_sheet = [requests.get('https://financialmodelingprep.com/api/v3/balance-sheet-statement/'+
                                 '{}?period=quarter&limit={}&apikey={}'.format(symbol, limit, API_KEY)) for symbol in sp500_list]

#converting the json object for each ticker
balance_sheet = [statement.json() for statement in balance_sheet]

#turning the list of json outputs into a dataframe
balance_sheet = pd.concat(pd.DataFrame(balance_sheet[i]) for i in range(len(balance_sheet)))

#converting the items in the date column into datetime objects
balance_sheet['date'] = [dt.datetime.strptime(date, "%Y-%m-%d") for date in balance_sheet['date']]

#appending the period and date column to be combined with symbol in order to create a multi index
balance_sheet.reset_index(inplace=True)
balance_sheet['year'] = [balance_sheet['date'][i].strftime("%Y") for i in range(len(balance_sheet['period']))]

#setting the multi index
balance_sheet.set_index(['symbol','year', 'period'], inplace=True)

In [105]:
statements_df = pd.merge(income_statement, balance_sheet, how='inner', left_index=True, right_index=True)

repeat steps to add enterprise value, then start buidling ratios

In [72]:
def quarter(date):

  '''
  This function takes in a series of dates, then converts them to month and day
  values, to be converted into fiscal quarters.
  '''

  date=date.strftime('%m-%d')

  if date >='01-01' and date <= '03-31':
    return 'Q1'
  elif date >= '04-01' and date <= '06-31':
    return 'Q2'
  elif date >= '07-01' and date <= '09-30':
    return 'Q3'
  else:
    return 'Q4'

In [93]:
enterprise_value = [requests.get('https://financialmodelingprep.com/api/v3/enterprise-values/{}?period=quarter&limit={}&apikey={}'.format(symbol, limit, API_KEY)) for symbol in sp500_list]

#converting the json object for each ticker
enterprise_value = [value.json() for value in enterprise_value]

#turning the list of json outputs into a dataframe
enterprise_value = pd.concat(pd.DataFrame(enterprise_value[i]) for i in range(len(enterprise_value)))

#converting the items in the date column into datetime objects
enterprise_value['date'] = [dt.datetime.strptime(date, "%Y-%m-%d") for date in enterprise_value['date']]

#creting the period column
enterprise_value['period'] = enterprise_value['date'].apply(quarter)

#appending the period and date column to be combined with symbol in order to create a multi index
enterprise_value.reset_index(inplace=True)
enterprise_value['year'] = [enterprise_value['date'][i].strftime("%Y") for i in range(len(enterprise_value['period']))]

#setting the multi index
enterprise_value.set_index(['symbol', 'year', 'period'], inplace=True)

In [94]:
enterprise_value

index       date  ...  addTotalDebt  enterpriseValue
symbol year period                    ...                               
MMM    2021 Q1          0 2021-03-31  ...  1.818700e+10     1.254017e+11
       2020 Q4          1 2020-12-31  ...  1.879500e+10     1.151547e+11
            Q3          2 2020-09-30  ...  1.959800e+10     1.080290e+11
            Q2          3 2020-06-30  ...  2.076200e+10     1.065492e+11
            Q1          4 2020-03-31  ...  2.249500e+10     9.698097e+10
...                   ...        ...  ...           ...              ...
ZTS    2017 Q4         15 2017-10-01  ...  4.952000e+09     3.436150e+10
            Q3         16 2017-07-02  ...  3.819000e+09     3.407298e+10
            Q2         17 2017-04-02  ...  3.718000e+09     2.942410e+10
       2016 Q4         18 2016-12-31  ...  4.468000e+09     3.018118e+10
            Q4         19 2016-10-02  ...  4.467000e+09     2.988604e+10

[9995 rows x 8 columns]

In [106]:
statements_df = pd.merge(statements_df, enterprise_value, how='left', left_index=True, right_index=True)

In [107]:
statements_df.columns

Index(['index_x', 'date_x', 'reportedCurrency_x', 'fillingDate_x',
       'acceptedDate_x', 'revenue', 'costOfRevenue', 'grossProfit',
       'grossProfitRatio', 'researchAndDevelopmentExpenses',
       'generalAndAdministrativeExpenses', 'sellingAndMarketingExpenses',
       'otherExpenses', 'operatingExpenses', 'costAndExpenses',
       'interestExpense', 'depreciationAndAmortization', 'ebitda',
       'ebitdaratio', 'operatingIncome', 'operatingIncomeRatio',
       'totalOtherIncomeExpensesNet', 'incomeBeforeTax',
       'incomeBeforeTaxRatio', 'incomeTaxExpense', 'netIncome',
       'netIncomeRatio', 'eps', 'epsdiluted', 'weightedAverageShsOut',
       'weightedAverageShsOutDil', 'link_x', 'finalLink_x', 'index_y',
       'date_y', 'reportedCurrency_y', 'fillingDate_y', 'acceptedDate_y',
       'cashAndCashEquivalents', 'shortTermInvestments',
       'cashAndShortTermInvestments', 'netReceivables', 'inventory',
       'otherCurrentAssets', 'totalCurrentAssets', 'propertyPlantEquipm

Start calculating ratios

starting with EPS and Price/Earnings

In [108]:
statements_df['pe_ratio'] = statements_df['stockPrice']/statements_df['eps']

In [110]:
statements_df['ROE'] = statements_df['netIncome']/statements_df['totalStockholdersEquity']

In [ ]:
statements_df['ROE']

In [130]:
statements_df['EV/EBITDA'] = statements_df['enterpriseValue'] / statements_df['ebitda']

In [159]:
#ratios to use: period	priceEarningsRatio	priceEarningsToGrowthRatio	returnOnEquity	returnOnCapitalEmployed	enterpriseValueMultiple

In [186]:
list(statements_df[(statements_df['EV/EBITDA'] != np.inf) and (statements_df['EV/EBITDA'] != -(np.inf))].loc[(slice(None), '2020', slice(None)),'EV/EBITDA'].dropna())

ValueError: ignored

In [190]:
statements_df[statements_df['EV/EBITDA'] == -(np.inf)][['enterpriseValue', 'ebitda','EV/EBITDA']]

enterpriseValue  ebitda  EV/EBITDA
symbol year period                                    
BK     2016 Q2        -4.085115e+10     0.0       -inf
            Q3        -3.200255e+10     0.0       -inf
            Q4        -2.940926e+09     0.0       -inf
       2017 Q1        -5.921108e+09     0.0       -inf
            Q2        -9.695004e+09     0.0       -inf
            Q3        -7.465432e+09     0.0       -inf
            Q4        -1.963276e+10     0.0       -inf
       2018 Q1        -2.683563e+10     0.0       -inf
            Q2        -8.311047e+09     0.0       -inf
            Q3        -1.061896e+10     0.0       -inf
            Q4        -5.432346e+09     0.0       -inf
       2019 Q2        -9.425533e+09     0.0       -inf
            Q3        -2.152314e+10     0.0       -inf
            Q4        -3.675706e+10     0.0       -inf
       2020 Q1        -1.140250e+11     0.0       -inf
            Q2        -7.121114e+10     0.0       -inf
            Q2        -7.121114e+10     0.0       -inf
       2021 Q1        -8.580883e+10     0.0       -inf
CMA    2020 Q2        -7.698521e+08     0.0       -inf
       2021 Q1        -1.975400e+09     0.0       -inf
MTB    2020 Q2        -2.538761e+09     0.0       -inf
            Q3        -4.347759e+09     0.0       -inf
       2021 Q1        -9.621261e+09     0.0       -inf
NTRS   2016 Q2        -4.123964e+09     0.0       -inf
            Q3        -5.694946e+09     0.0       -inf
            Q4        -2.533235e+09     0.0       -inf
       2017 Q1        -3.971585e+09     0.0       -inf
            Q2        -5.146660e+09     0.0       -inf
            Q3        -5.838249e+09     0.0       -inf
            Q4        -9.605867e+09     0.0       -inf
       2019 Q4        -2.312057e+09     0.0       -inf
       2020 Q2        -1.217001e+10     0.0       -inf
            Q3        -1.136417e+10     0.0       -inf
       2021 Q1        -2.490056e+10     0.0       -inf
RF     2021 Q1        -2.149740e+09     0.0       -inf
SIVB   2020 Q2        -1.873610e+09     0.0       -inf
            Q3        -1.917451e+09     0.0       -inf
STT    2016 Q2        -6.609189e+10     0.0       -inf
            Q3        -6.691563e+10     0.0       -inf
            Q4        -3.204158e+10     0.0       -inf
       2017 Q1        -2.747176e+10     0.0       -inf
            Q2        -2.072981e+10     0.0       -inf
            Q3        -1.733903e+10     0.0       -inf
            Q4        -2.163298e+10     0.0       -inf
       2018 Q1        -3.528958e+10     0.0       -inf
            Q2        -3.712053e+10     0.0       -inf
            Q3        -2.574991e+10     0.0       -inf
            Q4        -4.010374e+10     0.0       -inf
       2019 Q1        -2.162173e+10     0.0       -inf
            Q2        -3.006281e+10     0.0       -inf
            Q3        -2.645213e+10     0.0       -inf
            Q4        -3.139718e+10     0.0       -inf
       2020 Q2        -4.558142e+10     0.0       -inf
            Q3        -3.550407e+10     0.0       -inf
            Q4        -8.109355e+10     0.0       -inf
       2021 Q1        -6.802008e+10     0.0       -inf
ZION   2015 Q4        -5.239740e+08     0.0       -inf

In [182]:
-(np.inf)

-inf

inf issue is calused by EBITDA being equal to 0